In [7]:
from ds1000de import DS1000DE
scope=DS1000DE("USB0")

r = scope.inst.query
w = scope.inst.write

In [8]:
from mhs5200 import MHS5200
port = "COM12" # Serial port.
signal_gen = MHS5200(port=port)

chan1 = signal_gen.channels[0]

SerialException: could not open port 'COM12': PermissionError(13, 'Access is denied.', None, 5)

In [13]:
grid_t = 12
grid_v = 8
cycles = 10

In [14]:
period = 1/chan1.frequency # Seconds
scale_val = period*cycles/grid_t

In [15]:
w(":TIMEBASE:SCALE {}".format(scale_val))

(36, <StatusCode.success: 0>)

In [16]:
scale_pct = 0.8
scale = chan1.amplitude/scale_pct/grid_v
w(":CHAN1:SCALE {}".format(scale))

(21, <StatusCode.success: 0>)

In [17]:
w(":CHAN1:OFFSET 0")

(17, <StatusCode.success: 0>)

In [18]:
class Measure(object):
    def __init__(self, scope):
        self.scope = scope
        
    @property
    def inst(self):
        return self.scope.inst
    
    @property
    def _kw_(self):
        return self.__class__.__name__.upper()
    
    
    def _write_(self, cmd, arg=None):
        if arg is None:
            self.inst.write(":{}:{}".format(self._kw_, cmd))
        else:
            self.inst.write(":{}:{} {}".format(self._kw_, cmd, arg))
    
    def _query_(self, cmd):
        return self.inst.query(":{}:{}?".format(self._kw_, cmd))
    
    def clear(self):
        self._write_("clear")

In [19]:
measurements = [
    'vpp',
    'vmax',
    'vmin',
    'vamplitude',
    'vtop',
    'vbase',
    'vaverage',
    'vrms',
    'overshoot',
    'preshoot',
    'frequency',
    'risetime',
    'falltime',
    'period',
    'pwidth',
    'nwidth',
    'pdutycycle',
    'ndutycycle',
    'pdelay',
    'ndelay',
]

In [20]:
def measurement_factory(measurement):
    def measure(self):
        value_raw = self._query_(measurement)
        return float(value_raw)
    return measure

In [21]:
for measurement in measurements:
    fcn = measurement_factory(measurement)
    prop = property(fcn)
    setattr(Measure, measurement, prop)

In [22]:
m = Measure(scope)

In [23]:
m.vmax

5.19

In [24]:
m.frequency

1.96

In [25]:
m.ndutycycle

0.49

In [26]:
m.period

0.51

In [27]:
chan1.amplitude=10

In [28]:
m.vpp

10.4

In [29]:
m.vamplitude

10.4

In [43]:
def Q(*args):
    return r(":{}?".format(":".join(args)))
def W(*args):
    kw = ":".join(args[:-1])
    val = args[-1]
    return w(":{} {}".format(kw, val))

In [37]:
Q("TRIGGER:MODE")

'EDGE'

In [39]:
Q("TRIGGER:EDGE:LEVEL")

'0.00e+00'

In [40]:
Q("TRIGGER:STATUS")

'RUN'

In [41]:
Q("TRIGGER:EDGE:SWEEP")

'AUTO'

In [44]:
W("TRIGGER:EDGE:SWEEP", "SINGLE")

(28, <StatusCode.success: 0>)

In [45]:
m.vamplitude

10.4

In [46]:
m.frequency

2.0

In [47]:
m.period

0.5